# UE02 - RDF and RDF Schema

Before you start with this notebook, complete the eight tasks in the `1. RDF` sheet of `SemAI.jar`. You will then make use of your solutions in this notebook. 

## Task 0: Preparation

Preparation (Installs and Imports). 




In [ ]:
# Install required packages in the current Jupyter kernel
!pip install -q rdflib 
!pip install -q pydot
!pip install -q owlrl

!pip install networkx pyvis

import rdflib
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace, Dataset
import networkx as nx
from pyvis.network import Network
import requests
from IPython.display import display, HTML, Image
import os
import pydot
import owlrl
from rdflib.namespace import FOAF , XSD , RDFS 

# A function to produce a graphical visualization of an RDF graph
def visualize_graph(g,base=None):

  def node_id(graph,term):
    if isinstance(term,Literal): return term.n3(graph.namespace_manager)
    else: return f"\"{term.n3(graph.namespace_manager)}\""

  def add_node(dg,g,t,base):
    if isinstance(t,URIRef):
      lbl = f"\"{t.n3(g.namespace_manager)}\""
      if(base): lbl = lbl.replace(base,"")
      if(len(lbl)>25): lbl = lbl[:12] + "..." +  lbl[-12:] 
      dg.add_node(pydot.Node( node_id(g,t), label=lbl ))
    if isinstance(t,Literal):
      dg.add_node(pydot.Node( node_id(g,t), label=t.n3(g.namespace_manager), shape="box"))
    if isinstance(t,BNode):
      dg.add_node(pydot.Node( node_id(g,t), label=""))    

  def add_edge(dg,g,s,p,o):
    dg.add_edge(pydot.Edge(node_id(g,s), node_id(g,o), label=f"\"{p.n3(g.namespace_manager)}\""))

  dg = pydot.Dot('my_graph', graph_type='digraph',layout='sfdp', splines='curved' )

  for subj in g.subjects(None,None): add_node(dg,g,subj,base)
  for obj in g.objects(None,None): add_node(dg,g,obj,base)
  for (s,p,o) in g: add_edge(dg,g,s,p,o)   

  display(Image(dg.create_png()))

## Task 1:  Improve interactive RDF graph visualization (1 pt)

Improve function `visualize_graph_pyvis` (from `V01_rdf.ipynb`) as follows:
- an optional `base` parameter
- abbreviate the labels of nodes and edges in the same way as in `visualize_graph`. 
- make sure that blank node IDs are not shown in the visualization. 

Optional features: 
- use different graphical forms for literals and URIs
- (add further improvements as you like)

Test the function with `rdf_str` and with your solution to task `0. Intro` in the `1. RDF` sheet in `SemAI.jar`.

In [ ]:
rdf_str = """
BASE   <http://example.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX schema: <http://schema.org/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX wd: <http://www.wikidata.org/entity/>
 
<bob#me>
   a foaf:Person ;
   foaf:knows <alice#me> ;
   schema:birthDate "1990-07-04"^^xsd:date ;
   foaf:topic_interest wd:Q12418 .
   
wd:Q12418
  dcterms:title "Mona Lisa" ;
  dcterms:creator <http://dbpedia.org/resource/Leonardo_da_Vinci> .

<http://data.europeana.eu/item/04802/243FA8618938F4117025F17A8B813C5F9AA4D619>
  dcterms:subject wd:Q12418 .
"""

g1 = Graph()
g1.parse(format="turtle", data=rdf_str)

res_intro = '''
@prefix rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix ex:   <http://www.ex.org/> .

ex:John a foaf:Person ;
 foaf:knows ex:Mary .
'''

g2 = Graph()
g2.parse(format="turtle", data=res_intro)

blank_node_graph = '''
@prefix rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix ex:   <http://www.ex.org/> .

ex:John foaf:knows [ 
 foaf:knows ex:Mary ;
 a foaf:Person 
] .
'''

g3 = Graph()
g3.parse(format="turtle", data=blank_node_graph)



def visualize_graph_pyvis(g, base=None):

    # Create the NetworkX graph
    nx_graph = nx.DiGraph()

    for s, p, o in g:
        nx_graph.add_edge(s, o, label=p)

    # Create a PyVis network graph
    pyvis_graph = Network(notebook=True, cdn_resources='in_line',bgcolor="#EEEEEE")
    ###pyvis_graph.barnes_hut()
    ###pyvis_graph.show_buttons(filter_=['physics'])

    pyvis_graph.from_nx(nx_graph)

    # Customize the node appearance
    for node in pyvis_graph.nodes:
        if isinstance(node["id"], Literal): 
          node["shape"] = "box"
        else:
          node["shape"] = "dot"
        if isinstance(node["id"], BNode):
          node["label"] = ""
        node["size"] = 10
        node["font"] = {"size": 10}
        node["label"] = node["id"].n3(g.namespace_manager)
        if base is not None and base in node["label"]:
          node["label"] = node["label"].replace(base, "")
        lbl = node["label"]
        if len(lbl) > 25:
          node["label"] = lbl[:12] + "..." + lbl[-12:] 

    # Customize the edge appearance
    for edge in pyvis_graph.edges:
        edge["width"] = 0.5
        edge["font"] = {"size": 8, "align": "middle"}
        edge["arrows"] = "to"
        edge["label"] = edge["label"].n3(g.namespace_manager)
        if len(edge["label"]) > 25:
          edge["label"] = lbl[:12] + "..." + lbl[-12:]

    # Define the HTML file name
    html_file = 'graph.html'    
    
    # Show the graph in the notebook
    pyvis_graph.show(html_file)

    # Check if the file exists
    if os.path.isfile(html_file):
        # Read the content of the HTML file
        with open(html_file, 'r') as file:
            html_content = file.read()
        # Display the HTML content in the notebook
        display(HTML(html_content))
    else:
        print(f"File not found: {html_file}")

visualize_graph_pyvis(g1, base="http://example.org/")
#visualize_graph_pyvis(g2, base="http://example.org/")
#visualize_graph_pyvis(g3, base="http://example.org/")





## Task 2:  Print RDF graph as HTML table (1 pt)

Implement a function `rdf2htmltable(g)` that 
- takes as parameter an rdflib.Graph 
- generates and displays an HTML table representing that graph with
  - one line per RDF statement 
  - three columns (subject, predicate, object) 
  - URIs should be shown in abbreviated form and be represented as links (`href=<full URI>`)

Test the function with `rdf_str` and with your solution to task `0. Intro` in the `1. RDF` sheet in `SemAI.jar`.

In [ ]:
def rdf2htmltable(g, base=None):

  def get_label(s):
    s = s.n3(g.namespace_manager)
    if base is not None:
      s = s.replace(base, "")
    s = s.replace(">", "&gt")
    s = s.replace("<", "&lt")
    if len(s) > 25:
          s = s[:12] + "..." + s[-12:]
    return s

  table = '''<table>
              <tbody>
                <thead>
                  <tr>
                    <th>Subject</th>
                    <th>Predicate</th>
                    <th>Object</th>
                  </tr>
                </thead>
          '''
  for s, p, o in g:
    table += "<tr>"
    table += "<td>" + "<a href=\"" + s.n3()[1:-1] + "\">" + get_label(s) + "</a>" + "</td>"
    table += "<td>" + "<a href=\"" + p.n3()[1:-1] + "\">" + get_label(p) + "</a>" + "</td>"
    table += "<td>" + "<a href=\"" + o.n3()[1:-1] + "\">" + get_label(o) + "</a>" + "</td>"
    table += "</tr>\n"
  table += "</tbody></table>"
  return table

display(HTML(rdf2htmltable(g1, base="http://example.org/")))

## Task 3: A function for parsing and displaying an RDF graph (1 pt)

A function `parse_display_rdf(str)` that takes as parameter a string which represents an RDF graph in Turtle notation and 
- produces an rdflib.Graph from that string
- prints the graph in Turte notation
- prints the graph in RDF/XML
- visualizes it using `visualize_graph` (to be taken from `V01_rdf.ipynb`)
- visualizes it using (your improved version of) `visualize_graph_pyvis`
- outputs it using `rdf2htmltable` (only if you have implemented this function)

Test the function with `rdf_str` and with your solution to task `0. Intro` in the `1. RDF` sheet in `SemAI.jar`.

In [ ]:
def parse_display_rdf(str):
  g = Graph()
  g.parse(format="turtle", data=str)
  print("Turtle Format:")
  print(g.serialize(format="turtle"))
  print("RDF/XML Format:")
  print(g.serialize(format="xml"))
  print("Visualization :")
  visualize_graph(g)
  print("Visualization using improved pyvis:")
  visualize_graph_pyvis(g)
  print("HTML Table:")
  display(HTML(rdf2htmltable(g)))

parse_display_rdf(res_intro)


# Tasks 4-10 (1 point each)

For each of the remaining 7 tasks in the "1. RDF" sheet in `SemAI.jar` do the following: 
- add a text cell in this notebook 
  - with the description of the task from `SemAI.jar` 
  - with number and title (e.g., **1. Simple Data Graph**)  from `SemAI.jar` as header 
- add a code cell where you apply `parse_display_rdf(str)` on your solution

# 1. Simple Data Graph

Create an RDF graph in Turtle notation. Use the FOAF vocabulary to state the following.

    Mary and Jim are persons.
    Mary knows Jim.
    Mary is 27 years old. 

The URIs for the two persons should be http://www.ex.org/person#Mary and http://www.ex.org/person#Jim. The age of Mary should be represented as an integer. 

In [ ]:
parse_display_rdf(
'''
@prefix rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd:  <http://www.w3.org/2001/XMLSchema#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix :     <http://www.ex.org/person#> .

:Mary a foaf:Person.
:Jim a foaf:Person.

:Mary foaf:knows :Jim ;
 foaf:age "27"^^xsd:int.
'''
)

# 2. Simple Schema

Create a vocabulary using RDFS in Turtle. Specify

    Classes Company, Employee, and Person
    Property worksFor between Employee and Company
    Property salary of Employee with Integer as data type
    Class Employee is a subclass of Person 

Use XSD for data types. The URIs of classes and properties are in namespace <http://www.ex.org/vocabulary#>, for example, <http://www.ex.org/vocabulary#Company>

In [ ]:
parse_display_rdf(
'''
@prefix rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd:  <http://www.w3.org/2001/XMLSchema#> .
@prefix :     <http://www.ex.org/vocabulary#> .

:Employee a rdfs:Class ;
 rdfs:subClassOf :Person .

:Company a rdfs:Class .

:Person a rdfs:Class .

:salary a rdf:Property ;
 rdfs:domain :Employee ;
 rdfs:range xsd:integer .

:worksFor a rdf:Property ;
 rdfs:domain :Employee ;
 rdfs:range :Company .
'''
)

# 3. Reification

Create an RDF graph in Turtle notation. Use the RDF vocabulary where applicable and the example namesspace (ex:) for all other resources (ex:Mary, ex:John, ex:says, ex:loves). Hint: the lecture slides contain a similar reification example.

    Mary says that John loves her.  

In [ ]:
parse_display_rdf(
'''
@prefix rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix ex:   <http://www.ex.org/> .

ex:Statement rdf:type rdf:Statement .

ex:Mary ex:says ex:Statement .

ex:Statement rdf:subject ex:John .
ex:Statement rdf:predicate ex:loves .
ex:Statement rdf:object ex:Mary .
'''
)

# 4. Blank Node
Create an RDF graph in Turtle notation. Use the RDF and FOAF vocabularies where applicable and the example namesspace (ex:) for the other resources.

    John knows a person, who knows Mary.
    Use a blank node to represent that anonymous person. 

In [ ]:
parse_display_rdf(
'''
@prefix rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix ex:   <http://www.ex.org/> .

ex:John foaf:knows [ 
 foaf:knows ex:Mary ;
 a foaf:Person 
] .
'''
)

# 5. Multiple Classification

Create an RDF graph in Turtle notation. Use the RDF vocabulary where applicable and the example namesspace (ex:) for the other resources.

    John is an instance of SoccerPlayer and of Student. 

In [ ]:
parse_display_rdf(
'''
@prefix rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix ex:   <http://www.ex.org/> .

ex:John a ex:SoccerPlayer ;
 a ex:Student .
'''
)

# 6. Metamodeling

Create an RDF graph in Turtle notation. Use the RDF and RDF Schema vocabularies where applicable and the example namesspace (ex:) for the other resources.

    Dog and Cat are instances of Species and subclasses of Animal.
    Lassie is an instance of Dog. 

In [ ]:
parse_display_rdf(
'''
@prefix rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix ex:   <http://www.ex.org/> .

ex:Dog a ex:Species ;
 rdfs:subClassOf ex:Animal.

ex:Cat a ex:Species ;
 rdfs:subClassOf ex:Animal.

ex:Lassie a ex:Dog .
'''
)

# 7. Properties
Create an RDF graph in Turtle notation. Use the RDF and RDF Schema vocabularies where applicable and the example namesspace (ex:) for the other resources, e.g., ex:childOf, ex:descendantOf.

    Everyone who is a child of someone, is also a descendant of that someone. 

In [ ]:
parse_display_rdf(
'''
@prefix rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix ex:   <http://www.ex.org/> .

ex:childOf rdfs:subPropertyOf ex:descendantOf .
'''
)